In [1]:
using Random, Distributions
using Plots
# plotlyjs()
# using PlotlyJS
using DataFrames
using StatsBase
using DelimitedFiles
using LinearAlgebra
# using Distances
# using Symbolics
# using Latexify

# Problem Set 5 (Jonathan Fischer using Julia)

## 1) Langevin Dynamics of minimalist HP 

### Initialize Positions R

In [10]:
function initR()
    R = zeros((5,3))
    R[:,3] .= 1,2,3,4,5
    R
end

initR (generic function with 1 method)

In [11]:
initR()

5×3 Matrix{Float64}:
 0.0  0.0  1.0
 0.0  0.0  2.0
 0.0  0.0  3.0
 0.0  0.0  4.0
 0.0  0.0  5.0

### Calculate distance (single component)

In [17]:
function calc_dist(x1,x2) #calculate distance between a single pair of points along single component
    x1-x2 
    # dx - L*round(dx/L) #distance using minimum image convention 
end

calc_dist (generic function with 2 methods)

### Calculate non-bonded potential 

In [18]:
function calc_LJ(r, epsilon, sigma = 1) # calculate Lennard_Jones potential for a pairwise distance
    4*epsilon*((sigma/r)^12 - (sigma/r)^6) 
end

calc_LJ (generic function with 2 methods)

### Calculate force (dU + friction + noise) returns tuple of force terms

In [35]:
function calc_LJforce(r, epsilon, T, v, dt, sigma = 1, k=1, m=1, eta=0.05)
    # (48*epsilon*((sigma^12/r^13)-0.5*(sigma^6/r^7)) + randn()*√(2kT*m*eta/dt) - eta*m*v, 
    [48*epsilon*((sigma^12/r^13)-0.5*(sigma^6/r^7)),
    randn()*√(2kT*m*eta/dt),
    -(eta*m*v)]
end

calc_LJforce (generic function with 5 methods)

### Calculate bonded potential

In [79]:
function calc_harmU(r,k=20,l=1)
    0.5k*(r-l)^2
end

calc_harmU (generic function with 3 methods)

### Calculate bond force

In [80]:
function calc_harmforce(r,k=20,l=1)
    [k*(r-l),0,0]
end

calc_harmforce (generic function with 3 methods)

### Update acceleration matrix A

In [84]:
function update_A(A, R, T, V, dt, m = 1) #update distance matrix D and acceleration matrix A 
    PE = 0 #initialize potential energy
    N = length(R[:,1])
    A = zeros((N,3,3))
    for i in 1:N-1
        for j in i+1:N
            dr = calc_dist.(R[i,:],R[j,:]) #calculate distance vector between two points
            r_mag = norm(dr) #calculate magnitude of distance vector
            if j - i > 1
                if (i,j) == (1,4) || (i,j) == (1,5) || (i,j) == (4,5)
                    epsilon = 1.
                    PE += calc_LJ(r_mag, epsilon)  #accumulate LJ potential energy
                    # f_scalars = calc_LJforce(r_mag,epsilon,T,v,dt) #scalar magnitude of force
                    A[i,:,:] += calc_LJforce(r_mag,epsilon,T,norm(V[i]),dt)  * dr / r_mag #force vector with each component
                    A[j,:,:] -= calc_LJforce(r_mag,epsilon,T,norm(V[j]),dt)  * dr / r_mag #force vector with each component
                else
                    epsilon = 2/3
                    PE += calc_LJ(r_mag, epsilon)  #accumulate LJ potential energy
                    A[i,:,:] += [x * dr / r_mag for x in calc_LJforce(r_mag,epsilon,T,norm(V[i]),dt)] #force vector with each component
                    A[j,:,:] -= [x * dr / r_mag for x in calc_LJforce(r_mag,epsilon,T,norm(V[j]),dt)] #force vector with each component
                end
            else 
                PE += calc_harmU(r_mag)
                fvector = [x * dr/ r_mag for x in calc_harmforce(r_mag)]
                A[i,:,:] += fvector
                A[j,:,:] -= fvector
            end
        end
    end
    return PE, A
end

update_A (generic function with 2 methods)

In [86]:
r_mag = 2
dr = [1,2,3]

fvector = [x * dr/ r_mag for x in calc_harmforce(r_mag)]


3-element Vector{Vector{Float64}}:
 [10.0, 20.0, 30.0]
 [0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0]

In [87]:
calc_harmforce(r_mag)

3-element Vector{Int64}:
 20
  0
  0

### Calculate new position {vector}

In [63]:
function calc_position(r, v, a, dt) #update position vector for single particle
    r + (v*dt) + 0.5(a*dt^2)
end

calc_position (generic function with 1 method)

### Update position matrix R in place (V,A,dt)

In [64]:
function update_R!(R, V, A, dt) #updates position matrix R IN PLACE
    broadcast!(calc_position,R,R,V,A,dt) #vectorized operation passed R by reference 
end

update_R! (generic function with 1 method)

### Calculate velocity {v::float, a1::float, a2::float, dt::float} with BBK

In [65]:
function calc_velocity(v,a1,a2,dt,eta=0.05) #velocity verlet update of velocity for single component
    (1+dt*eta/2)^-2 * (v + 0.5(a1)*dt + 0.5(a2[1] + a2[2])*dt) #average current and new accelerations
end

calc_velocity (generic function with 2 methods)

### Update velocity matrix V (V,A1,A2,dt)

In [66]:
function update_V!(V,A1,A2,dt) #updates velocity matrix V IN PLACE (passed by reference)
    broadcast!(calc_velocity,V,V,A1,A2,dt)
end

update_V! (generic function with 1 method)

### Initialize velocity (T,N,m,k)

In [67]:
function init_velocity(T, N, m=1, k=1) #temperature, count, and mass
    V = √(k*T/m)*randn((N,3)) #return N x 3 matrix of initialized velocities sampled from Maxwell-Boltzmann distribution
    V[end,:] = -sum(V[1:end-1,:],dims=1) #Momentum conservation: last row equals minus sum of all other rows. Mass cancels out because same species 
    return V
end

init_velocity (generic function with 3 methods)

### Calculate KE and T

In [68]:
function calc_KE(V, N, m =1)
    KE = 0.
    T = 0.
    for p in eachrow(V)
        v_squared = sum(p.^2) #get squared magnitude of velocity vector
        KE += 0.5(m*v_squared) #accumulate KE for each particle
        T += v_squared/(3N-3)
    end
    return KE, T 
end

calc_KE (generic function with 2 methods)

### LD Simulation

In [82]:
function LD(T, dt, iters, R=initR())

    N = length(R[:,1]) #number of particles
    V = init_velocity(T,N) #velocity for t
    A = zeros(N,3,3) #acceleration matrix
    
    PE = Vector{Float64}(undef,iters+1) #initialize array of potential energies 
    PE[1], A = update_A(A,R,T,V,dt) #initialize acceleration matrix and return initial PE
    

    KE = Vector{Float64}(undef,iters+1) #initialize array of kinetic energies 
    KE[1], = calc_KE(V,N) #initial KE 

    T_list = Vector{Float64}(undef,iters+1) #initialize array of kinetic energies 
    T_list[1] = T #initial temperature


    for i in 1:iters
        curr_A = sum(copy(A),dims=3) #A for current t
        update_R!(R,V,curr_A,dt) #new R updated to t+dt
        PE[i+1],A = update_A(A, R,T,V,dt) #new A updated in-place to t+dt, and returns PE into array 
        update_V!(V,curr_A,A,dt) #new V updated to t+dt
        KE[i+1],T_list[i+1] = calc_KE(V,N) #add KE
        # T_list[i+1] = calc_T(KE[i+1],kb,N) #add T
    end
    return  PE, KE, T_list, A, V, R 
end

LD (generic function with 2 methods)

In [85]:
# Random.seed!(1234)
PE, KE, T_list, A, V, R= LD(2.,0.003,10000)

LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(3)), must have singleton at dim 2